#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [1]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Скачаем данные

In [2]:
# ! rm -rf data

In [3]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

replace __MACOSX/._data? [y]es, [n]o, [A]ll, [N]one, [r]ename: total 12
drwx------ 4 root root 4096 Nov 26 08:32 data
drwxr-xr-x 3 root root 4096 Dec 19 15:06 __MACOSX
drwxr-xr-x 1 root root 4096 Dec 16 00:01 sample_data


Посмотрим как выглядит один из файлов разметки

In [2]:
!cat data/train/boxes/apple_2.xml

<annotation>
	<folder>train</folder>
	<filename>apple_2.jpg</filename>
	<path>C:\tensorflow1\models\research\object_detection\images\train\apple_2.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>250</width>
		<height>250</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>37</xmin>
			<ymin>52</ymin>
			<xmax>205</xmax>
			<ymax>222</ymax>
		</bndbox>
	</object>
</annotation>


## Релизуйте выборку для YoloV1 - 2 балла

In [3]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc

import collections

from pathlib import Path
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [4]:
import os
import shutil

# Moving files in cetain directories
sourcepath='./data/train'
sourcefiles = os.listdir(sourcepath)
destinationpath = './data/train/images'
for file in sourcefiles:
    if file.endswith('.jpg'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))


sourcepath='./data/train'
sourcefiles = os.listdir(sourcepath)
destinationpath = './data/train/boxes'
for file in sourcefiles:
    if file.endswith('.xml'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

sourcepath='./data/test'
sourcefiles = os.listdir(sourcepath)
destinationpath = './data/test/images'
for file in sourcefiles:
    if file.endswith('.jpg'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

sourcepath='./data/test'
sourcefiles = os.listdir(sourcepath)
destinationpath = './data/test/boxes'
for file in sourcefiles:
    if file.endswith('.xml'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

In [5]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __len__(self):
          return len(self.box_paths)
    def __init__(self, data_dir, transforms=None):
      self.image_paths = []
      self.box_paths = []

      for root, dirs, files in os.walk(os.path.abspath(data_dir + '/images')):
        for file in files:
            self.image_paths.append(os.path.join(root, file))

      for root, dirs, files in os.walk(os.path.abspath(data_dir + '/boxes')):
        for file in files:
            self.box_paths.append(os.path.join(root, file))
      
      
      self.image_paths.sort()
      self.box_paths.sort()

      assert len(self.image_paths) == len(self.box_paths)

      self.transforms = transforms

    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):              
      image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
      boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx])

      resbo = []
      for i, item in enumerate(boxes):
        resbo.append(self.__convert_to_yolo_box_params(item, len(image[i]), len(image)))

      if self.transforms:
        image = self.transforms(image=image, bboxes=resbo, class_labels=class_labels)["image"]

      target_tensor = torch.zeros((7, 7, 3 + 5 * 2))
      counter = 0
      for box in resbo:
        class_label = class_labels[counter]
        counter += 1
        class_label = class2tag[class_label]

        if (target_tensor[int(7 * box[0]), int(7 * box[1]), 3] == 0):
          target_tensor[int(7 * box[0]), int(7 * box[1]), 3] = 1 # first box on this place
          target_tensor[int(7 * box[0]), int(7 * box[1]), 4:8] = torch.tensor(box) # first box on this place
        else:
          target_tensor[int(7 * box[0]), int(7 * box[1]), 8] = 1 # if some object already here
          target_tensor[int(7 * box[0]), int(7 * box[1]), 9:13] = torch.tensor(box)

        target_tensor[int(7 * box[0]), int(7 * box[1]), class_label - 1] += 1 # we start from 1 (apples)

      return torch.reshape(torch.unsqueeze(torch.from_numpy(image).to("cpu"), 0), (3, 448, 448)).float(), target_tensor


    def __get_boxes_from_xml(self, xml_filename: str):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []
      
      with open(xml_filename) as fd:
        doc = xmltodict.parse(fd.read())

      try:
        for artikel in doc['annotation']['object']:
          class_labels.append(artikel['name'])
        for artikel in doc['annotation']['object']:
          lines = []
          lines.append(artikel['bndbox'])
          for line in lines:
            reser = []
            reser.append(int(line['xmin']))
            reser.append(int(line['ymin']))
            reser.append(int(line['xmax']))
            reser.append(int(line['ymax']))
            boxes.append(reser)
      except:
        class_labels.append(doc['annotation']['object']['name'])
        line = doc['annotation']['object']['bndbox']
        reser = []
        reser.append(int(line['xmin']))
        reser.append(int(line['ymin']))
        reser.append(int(line['xmax']))
        reser.append(int(line['ymax']))
        boxes.append(reser)

      return boxes, class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """
      ans = []

      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
    
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height

      return ans

In [6]:
WIDTH, HEIGHT = 448, 448

train_transform = A.Compose([
    A.Resize(width=WIDTH, height=HEIGHT, always_apply=True),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ],
    bbox_params=A.BboxParams(format='yolo',label_fields=['class_labels']))
test_transform = A.Compose([
    A.Resize(width=WIDTH, height=HEIGHT, always_apply=True),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ],
    bbox_params=A.BboxParams(format='yolo',label_fields=['class_labels']))

In [7]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test"
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][1], torch.Tensor)
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [8]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 4,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=4, 
    shuffle=False
    )

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [9]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """
    intersection_bbox = torch.stack(
        [
            torch.max(predicted_bbox[..., 0:1], gt_bbox[..., 0:1]),
            torch.max(predicted_bbox[..., 1:2], gt_bbox[..., 1:2]),
            torch.min(predicted_bbox[..., 2:3], gt_bbox[..., 2:3]),
            torch.min(predicted_bbox[..., 3:4], gt_bbox[..., 3:4]),
        ]
    )
    intersection = (intersection_bbox[2] - intersection_bbox[0]).clamp(0) * (intersection_bbox[3] - intersection_bbox[1]).clamp(0)

    box1_area = abs((predicted_bbox[..., 2:3] - predicted_bbox[..., 0:1]) *
                    (predicted_bbox[..., 3:4] - predicted_bbox[..., 1:2]))
    box2_area = abs((gt_bbox[..., 2:3] - gt_bbox[..., 0:1]) *
                    (gt_bbox[..., 3:4] - gt_bbox[..., 1:2]))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [10]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C

        ## YOUR CODE

        self.layers = nn.Sequential(
            CNNBlock(3, 64, True, kernel_size=7, stride=2, padding=3),
            CNNBlock(64, 192, True, kernel_size=3, stride=1, padding=1),

            CNNBlock(192, 128, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(128, 256, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(256, 256, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(256, 512, True, kernel_size=3, stride=1, padding=1),

            CNNBlock(512, 256, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(256, 512, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(512, 256, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(256, 512, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(512, 256, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(256, 512, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(512, 256, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(256, 512, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(512, 512, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(512, 1024, True, kernel_size=3, stride=1, padding=1),

            CNNBlock(1024, 512, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(512, 1024, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(1024, 512, False, kernel_size=1, stride=1, padding=0),
            CNNBlock(512, 1024, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(1024, 1024, False, kernel_size=3, stride=1, padding=1),

            CNNBlock(1024, 1024, False, kernel_size=3, stride=2, padding=1),

            CNNBlock(1024, 1024, False, kernel_size=3, stride=1, padding=1),
            CNNBlock(1024, 1024, False, kernel_size=3, stride=1, padding=1),
        )

        self.layers_last = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024*S*S, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, S*S*(C+B*5)),
        )

        
        ## YOUR CODE

    def forward(self, x):
        x = self.layers_last(self.layers(x))
        return x


# Убедитесь на одном изображении, что предсказания вашей модели имеют верное количество значений

#testing_model = YOLO()
#expected_output_shape = testing_model.S * testing_model.S * (5 * testing_model.B + testing_model.C)

#testing_image = train_dataset[0][0]

#assert testing_model(testing_image).reshape(-1).shape[0] == expected_output_shape

## Реализуйте YoloLoss - 3 балла

In [41]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, 7, 7, 3 + 2 * 5)

        iou_b1 = intersection_over_union(predictions[..., 4:8], target[..., 4:8]) # first box intersection
        iou_b2 = intersection_over_union(predictions[..., 9:13], target[..., 4:8]) # second box intersection
        iou_b3 = intersection_over_union(predictions[..., 4:8], target[..., 9:13]) # first box intersection
        iou_b4 = intersection_over_union(predictions[..., 9:13], target[..., 9:13]) # second box intersection
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0), iou_b3.unsqueeze(0), iou_b4.unsqueeze(0)], dim=0)
        iou_maxes, bestbox = torch.max(ious, dim=0)

        first_box = target[..., 3].unsqueeze(3)
        second_box = target[..., 8].unsqueeze(3)

         # check if box really exists in cell
        a = 0
        iser = 0
        if (first_box.nelement() != 0):
          iser = 1
          box_targets = target[..., 4:8]
          a = 1
        if (second_box.nelement() != 0):
          iser = 1
          box_targets = target[..., 9:13]
          a = 2

        box_predictions = iser * (
            (
                bestbox * predictions[..., 4:8]
                + (1-bestbox) * predictions[..., 9:13]
            )
        )

        # loss for width and height
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )

        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2)
        )

        # check if wrong object
        if (predictions[..., 3].nelement() != 0):
          pr_b = 1
        if (predictions[..., 8].nelement() != 0):
          pr_b = 1


        if (a == 1):
          q = torch.flatten(iser * target[..., 4:8] * iou_maxes)
        if (a == 2):
          q = torch.flatten(iser * target[..., 9:13] * iou_maxes)
        object_loss = self.mse(
            torch.flatten(iser * target[..., 3]),
            q
        )

        # loss in real object wasnt found
        no_object_loss = self.mse(
            torch.flatten((1 - iser) * predictions[..., 3], start_dim=1),
            torch.flatten((1 - iser) * target[..., 3], start_dim=1)
        )

        no_object_loss += self.mse(
            torch.flatten((1 - iser) * predictions[..., 8], start_dim=1),
            torch.flatten((1 - iser) * target[..., 8], start_dim=1)
        )

        # wrong class loss
        class_loss = self.mse(
            torch.flatten(iser * predictions[..., :3], end_dim=-2),
            torch.flatten(iser * target[..., :3], end_dim=-2)
        )
        

        loss = (
            self.lambda_coord * box_loss
            + object_loss
            + self.lambda_noobj * no_object_loss
            + class_loss
        )
        return loss

## Реализуйте дополнительные функции из статьи - 2 балла

In [37]:
def non_max_suppression(bboxes, iou_threshold, threshold):
    ## YOUR CODE
    pass

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    ## YOUR CODE
    pass

def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    ## YOUR CODE
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [33]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        x, y = train_batch
        logits = self(x)
        loss = self.loss(logits, y)
        return loss

    def validation_step(self, val_batch, batch_idx) -> None:
        x, y = val_batch
        logits = self(x)
        loss = self.loss(logits, y)
        return loss

In [ ]:
model = YOLO()
n_epochs = 10

yolo_learner = YOLOLearner()

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image